Importing Libraries

In [92]:
from tkinter import *
import tkinter as tk
import tkinter.ttk as ttk
import time
import cv2
import shutil
import numpy as np
import os
import serial
from PIL import ImageTk, Image

window for splash screen (loading screen when processing a papaya)

In [93]:

def splash():

    #Creates window
    w=Tk()
    width_of_window = 427
    height_of_window = 250
    screen_width = w.winfo_screenwidth()
    screen_height = w.winfo_screenheight()
    x_coordinate = (screen_width/2)-(width_of_window/2)
    y_coordinate = (screen_height/2)-(height_of_window/2)
    w.geometry("%dx%d+%d+%d" %(width_of_window,height_of_window,x_coordinate,y_coordinate))
    w.overrideredirect(1)
    w.wm_attributes("-topmost", 1)


    s = ttk.Style()
    s.theme_use('clam')
    s.configure("red.Horizontal.TProgressbar", foreground='red', background='#4f4f4f')
    progress=ttk.Progressbar(w,style="red.Horizontal.TProgressbar",orient=HORIZONTAL,length=500,mode='determinate',)

    def stop_buttons():
        print("Program is Stoping")
        w.destroy()

    #############progressbar          33333333333333333333333333333
    def new_win():
        w.destroy()
        q=Tk()
        q.title('Main window')
        q.geometry('427x250')
        l1=Label(q,text='ADD TEXT HERE ',fg='grey',bg=None)
        l=('Calibri (Body)',24,'bold')
        l1.config(font=l)
        l1.place(x=80,y=100)
        
        
        
        q.mainloop()



    def bar(w):
        
        global r
        l4=Label(w,text='Loading...',fg='white',bg=a)
        lst4=('Calibri (Body)',10)
        l4.config(font=lst4)
        l4.place(x=18,y=210)
        
       
        r=0
        progress['value']=r
        w.update_idletasks()
        time.sleep(0.03)
        l4=Label(w,text='Taking Photos',fg='white',bg=a)
        lst4=('Calibri (Body)',10)
        l4.config(font=lst4)
        l4.place(x=18,y=210)
        
        r=r+10
        progress['value']=r
        w.update_idletasks()
        main(w,progress,l4)
        time.sleep(0.03)
        r=r+10
        progress['value']=r
        w.update_idletasks()
        time.sleep(0.03)
        r=r+10
        progress['value']=r
        w.update_idletasks()
        time.sleep(0.03)
        r=r+10
        progress['value']=r
        w.update_idletasks()
        time.sleep(0.03)
        r=r+10
        progress['value']=r
        w.update_idletasks()
        time.sleep(0.03)
        r=r+10
        progress['value']=r
        w.update_idletasks()
        time.sleep(0.03)
        r=r+10
        """
        progress['value']=r
        w.update_idletasks()
        time.sleep(0.03)
        r=r+10
        progress['value']=r
        w.update_idletasks()
        time.sleep(0.03)
        r=r+10
        progress['value']=r
        w.update_idletasks()
        time.sleep(0.03)
        r=r+10
        """
        start_prog(w)
                
        
    progress.place(x=-10,y=235)

    a='#249794'
    x = Frame(w,width=427,height=241,bg=a)
    x.place(x=0,y=0)  #249794
    b1=Button(w,width=10,height=1,text='START',command=lambda :bar(w),border=0,fg=a,bg='white')
    b1.place(x=170,y=200)

    stop = tk.Button(w,text='Stop', padx= 20,command=stop_buttons)
    stop.grid(sticky=tk.NE)

    l1=Label(w,text='Processing',fg='white',bg=a)
    lst1=('Calibri (Body)',18,'bold')
    l1.config(font=lst1)
    l1.place(x=50,y=80)

    l2=Label(w,text='Papaya',fg='white',bg=a)
    lst2=('Calibri (Body)',18)
    l2.config(font=lst2)
    l2.place(x=185,y=82)

    l3=Label(w,text='Papaya Project',fg='white',bg=a)
    lst3=('Calibri (Body)',10)
    l3.config(font=lst3)
    l3.place(x=50,y=110)

    


    w.mainloop()


In [106]:
def process(model_disease):
    count = 0
    dirname = r'others\test'
    disease_list_final=  {'anthracnose':0,'black_spot':0,'phytophthora':0,'powdery_mildew':0,'ring_spot':0}
    colorlist = []
    percentage_list = []
    green_sum = 0
    yellow_sum = 0
    ran = 3

    for i in range (ran):
        print('######################################################')
        print('             Image               ' + str(i))
        
        path = os.path.join(dirname,  "papaya_images_{}".format(str(i) + '.png'))
        print(path)
        cam = cv2.imread(path)

        frame = cv2.resize(cam, (224,224))
        frame1 = np.array(frame)
        disease_img = frame
        maturity_img = frame1

        
        cv2.imwrite("PapayaImage.png", disease_img)

        disease_list= output(disease_img, model_disease)
        
        #show(frame, disease, percentage, maturity[0])
        green, yellow = detect_maturity(maturity_img)
        
        print("Image ",i, " Green: ", green, " Yellow: ", yellow)
        colorlist.append((green,yellow))

        green_sum += green
        yellow_sum += yellow
        
        for key in disease_list_final:
            if disease_list[key] > disease_list_final[key]:
                disease_list_final[key] = disease_list[key]

        print('######################################################')


        
    print("Disease list = " ,disease_list_final)
    print("G/Y total = " ,green_sum,yellow_sum)
    print("Ave Green = ", green_sum/ran)
    print("Ave Yellow = ", yellow_sum/ran)
    """
    for i in range(len(colorlist)):
        print("Image" , i , "=", colorlist[i])
        print(colorlist[i][0],colorlist[i][2])
        green_sum += colorlist[i][0]
        yellow_sum += colorlist[i][2]
    """
    

    


    green = green_sum/ran
    yellow = yellow_sum/ran
    print("Diseases : ", disease_list,percentage_list)
    print("Green: ", green)
    print("Yellow: ",yellow)


    if (green > 84) & (yellow <16):
        Maturity = "green"
    elif (green > 75) & (yellow < 25):
        Maturity = "breaker"
    elif (green > 66) & (yellow < 33):
        Maturity = "1/3 ripe"
    elif (green > 50) & (yellow < 50):
        Maturity ="half ripe"
    elif (green > 20) & (yellow < 80):
        Maturity = "ripe"
    else:
        Maturity =  "table ripe"

    return (Maturity,disease_list_final)
        

In [94]:
def main_program():
    global img_counter, dirname1
    

    #Delete existing directory 'test'
    shutil.rmtree(dirname1)

    #Creates directory 'test' 
    os.mkdir(dirname1)

    img_counter = 0
    #Arduino.write(b'E')


    #Starts taking pictures
    while True:
        ret, frame = cam.read()

        #Resize the frame(image captured) to 224x224
        frame = cv2.resize(frame, (224,224))
        #assign fram ti
        disease_img = frame
        frame = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        

        capture(frame)
        
        frame1 = np.array(frame)
        maturity_img = cv2.cvtColor(frame1,cv2.COLOR_BGR2RGB)

        #confirm_papaya = (detect_papaya(disease_img,model_detect))
        #print('CONFIRM PAPAYA: ', confirm_papaya)


        img_list = []
        #roll()

        cv2.imwrite("PapayaImage.png", disease_img)
        if img_counter>3:
            #Arduino.write(b'D')
            cam.release()
            break


In [95]:
def capture(img):
        global img_counter, dirname1
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        img_name = "papaya_images_{}.png".format(img_counter)
        cv2.imwrite(os.path.join(dirname1, img_name), img)
        print("Image Taken ", [img_counter])
        img_counter += 1

In [103]:
def start_prog(w):
    #main, model_detect, model_disease,disease_list_final,maturity_detected,weight
    #main, model_detect, model_disease,disease_list_final,maturity,weight
    print('Loading start_prog..')
    global cam
    """"""
    w.destroy()
  
    folder = r'test'
    weight = 100
    maturity_detected = "Table Ripe"
    disease_list_final=  {'anthracnose':0,'black_spot':0,'phytophthora':0,'powdery_mildew':0,'ring_spot':0}
    
    disease = "antracnose"
    path = os.listdir(folder)
    print(path)
    newpath = os.path.join(folder,(path[index1]))
    print(newpath)

    """
    def update_weight():
        Arduino.write(b'R')
        weight = Arduino.readline().decode('utf-8').rstrip()
    
    update_weight()
    """
    
    
    main = tk.Tk()
    width_of_window = 480
    height_of_window = 320
    screen_width = main.winfo_screenwidth()
    screen_height = main.winfo_screenheight()
    x_coordinate = (screen_width/2)-(width_of_window/2)
    y_coordinate = (screen_height/2)-(height_of_window/2)
    main.geometry("%dx%d+%d+%d" %(width_of_window,height_of_window,x_coordinate,y_coordinate))

    main.configure(background='black')
    main.title('Papaya Project')
    main.resizable(width = False, height= False)
    main.overrideredirect(True)
    main.wm_attributes("-topmost", 1)
    

    def restart_buttons(main,model_detect, model_disease):
        main.destroy()
        #main_start(model_detect, model_disease)

    def stop_buttons(main):
        main.destroy()
        

    header = tk.Frame(main,width=480, height = 40, bg = '#009ea0',bd = 0)
    header.grid(row = 0, columnspan=6 , column = 0)
    
    restart = tk.Button(main,text='Restart',command=lambda: restart_buttons(main, model_detect, model_disease), padx= 20)
    restart.grid(row = 0, column = 3 ,sticky=tk.W)

    stop = tk.Button(main,text='Stop', padx= 20,command=lambda : stop_buttons(main))
    stop.grid(row = 0, column = 4,columnspan=2 ,sticky=tk.W)

    body = tk.Frame(main,width=480, height = 250, bg = 'lightblue',bd = 0)
    body.grid(row = 1,column = 0, columnspan= 6, rowspan=4)

    imageframe = tk.Frame (main, width= 260, height= 250, bg = 'white' )
    imageframe.grid(row = 1, column=0, sticky= tk.W, columnspan=3, rowspan =4 )

    #outputframe = Frame(s, width=220, height=250, bg = 'red')
    #outputframe.grid(row = 1, rowspan=4, column=3, columnspan=3)

    output = tk.Label(main,text = 'Output',bg = 'lightblue',fg = 'black',bd = 0 , pady=10, font =('Times',15,'bold'))
    output.grid(row = 1, columnspan= 3, rowspan=2, column= 3, sticky = tk.N)
    #output.place(x = 300, y = 280)

    maturity_text = tk.Label(main, text ='Maturity Level : ', bg = 'lightblue' ,fg = 'black',pady=10, font =('Times',10))
    maturity_text.grid(row = 2, column= 3 , columnspan= 2, sticky=tk.W )

    

    maturity_bg= tk.Frame(main, width=60, height= 40, bg = '#333333')
    maturity_bg.grid(row = 2, column= 4 , columnspan= 3, sticky=tk.W )
    

    maturity = tk.Label(maturity_bg, text =maturity_detected, bg = '#DDDDDD' ,fg = '#222222',pady=10, font =('Times',10))
    maturity.grid(row = 2, column= 4 , columnspan= 2, sticky=tk.W )

    disease_text = tk.Label(main, text ='Disease Detected : ', bg = 'lightblue' ,fg = 'black',pady=10)
    #disease_text.grid(row = 3, column= 3 , columnspan= 2, sticky=tk.NW )
    #disease_text = tk.Label(main, text="\n".join([f"{key}: {value}" for key, value in disease_list_final.items()]))
    disease_text.grid(row = 3, column= 3 , columnspan= 2, sticky=tk.NW )

    disease_bg= tk.Frame(main, width=60, height= 40, bg = '#333333')
    disease_bg.grid(row = 3, column= 4 , columnspan= 3, sticky=tk.NW )

    disease = tk.Label(main, text="\n".join([f"{key}: {value}" for key, value in disease_list_final.items()]))
    disease.grid(row = 3, column= 4 , columnspan= 2, sticky=tk.NW )
    

    size_text = tk.Label(main, text ='Weight : ', bg = 'lightblue' ,fg = 'black',pady=10)
    size_text.grid(row = 4, column= 3 , columnspan= 2, sticky=tk.NW )

    size_bg= tk.Frame(main, width=60, height= 40, bg = '#333333')
    size_bg.grid(row = 4, column= 4 , columnspan= 3, sticky=tk.NW )

    size = tk.Label(size_bg, text =str(weight) + 'grams' , bg = '#DDDDDD' ,fg = '#222222',pady=10, font =('Times',10))
    size.grid(row = 4, column= 4 , columnspan= 2, sticky=tk.NW )

    #body.place(x = 0, y = 50)
    
    footer = tk.Frame(main,width=480, height = 35, bg = 'white',bd = 0)
    footer.grid(row = 5,column=0 ,columnspan= 6, rowspan=3, sticky = tk.N)

    
    
    
    
    #navigate = Frame(s,width=150, height=20,bg = 'blue',bd = 0 )
    #navigate.grid(row = 6, columnspan= 3, column= 0, sticky= NW,padx = 20)
    #navigate.place(x = 65 ,y = 300)
    """
    logo = Image.open(r'papaya_logo.png')
    logo = logo.resize((40,30))
    logo = ImageTk.PhotoImage(logo)
    image_label = tk.Label(main,image=logo, bd = 0,bg = 'lightblue')
    image_label.image = logo
    image_label.grid(row = 0, column = 0,columnspan=4 ,sticky=tk.E)
    image_label.place(x = 5, y = 5)
    """
   
    title = tk.Label(main, text  = 'Papaya \nProject', bd = 0, fg ='#343434'  ,
                font=('sans',9,'bold'), justify = 'left', bg = '#009ea0', pady=5)
    title.grid(row = 0, column=1, columnspan=5)
    title.place(x = 60)

    """
    
    path = os.listdir(folder)
    print(path)
    newpath = os.path.join(folder,(path[index1]))
    print(newpath)"""
    
    img = Image.open(newpath)
    img = img.resize((244,244))
    img = ImageTk.PhotoImage(img)
    imglabel = tk.Label(imageframe,image = img)
    imglabel.image = img
    imglabel.grid(row = 1, column=0, sticky= tk.W, columnspan=3, rowspan =4)
    #imglabel.place(x= 5, y = 55)
   
    

    def rightbutton(newpath,path,img, imglabel):
        global index1
        if index1 < len(path) - 1:
            index1 += 1

        print(index1)
        newpath  = os.path.join(folder,(path[index1]))
        img = Image.open(newpath)
        img = img.resize((244,244))
        img = ImageTk.PhotoImage(img)
        imglabel = tk.Label(main,image = img)
        imglabel.image = img
        imglabel.grid(row = 1, column=0, sticky= tk.W, columnspan=3, rowspan =4)

        items = tk.Label(main, text = 'Image %i of %i' %(index1+1, len(path)), padx = 10)
        items.grid(row=6, column=1)

    def leftbutton(newpath,path,img, imglabel):
        global index1
        
        if index1 > 0:
            index1 -= 1

        print(index1)
        newpath  = os.path.join(folder,(path[index1]))
        
        print(newpath)
        imglabel.grid_forget()
        
        img = Image.open(newpath)
        img = img.resize((244,244))
        img = ImageTk.PhotoImage(img)
        imglabel = tk.Label(main,image = img)
        imglabel.image = img
        imglabel.grid(row = 1, column=0, sticky= tk.W, columnspan=3, rowspan =4)

        items = tk.Label(main, text = 'Image %i of %i' %(index1+1, len(path)), padx = 10)
        items.grid(row=6, column=1)




    right = tk.Button(main, text = ">", command = lambda : rightbutton(newpath,path,img, imglabel),padx = 10 , bg ='#009ea0')
    left = tk.Button(main, text = "<", command = lambda : leftbutton(newpath,path,img, imglabel),padx = 10, bg ='#009ea0' )

    right.grid(row = 6, column = 2 ,sticky=tk.NW)
    #right.place(y = 258, x= 410 )
    left.grid(row = 6, column=0, sticky=tk.NE)
    #left.place(y = 258, x= 300 )

    items = tk.Label(main, text = 'Image %i of %i' %(index1+1, len(path)), padx = 10)
    items.grid(row=6, column=1)


    
  
    main.mainloop()

In [97]:
def main(w, progress,l4):
    global Arduino, cam, r, a
    
    #Delete existing directory 'test'
    shutil.rmtree(dirname1)

    #Creates directory 'test' 
    os.mkdir(dirname1)

    #Arduino.write(b'E')
    time.sleep(2)

    for i in range(3):
        c=0
        ret, frame = cam.read()

        #Resize the frame(image captured) to 224x224
        frame = cv2.resize(frame, (224,224))
        #assign fram ti
        disease_img = frame
        frame = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        

        capture(frame)
        
        frame1 = np.array(frame)
        maturity_img = cv2.cvtColor(frame1,cv2.COLOR_BGR2RGB)

        #confirm_papaya = (detect_papaya(disease_img,model_detect))
        #print('CONFIRM PAPAYA: ', confirm_papaya)
        l4.destroy()
        l4=Label(w,text="image '"+ str(i+1) + "'of 4",fg='white',bg=a)
        lst4=('Calibri (Body)',10)
        l4.config(font=lst4)
        l4.place(x=18,y=210)
        

        time.sleep(0.03)
        r=r+10
        progress['value']=r
        w.update_idletasks()
        

        img_list = []
        """
        Arduino.write(b'H')
        print('High')
        line  = Arduino.readline().decode('utf-8').rstrip()
        
        
        
        print(line)
        while True:
            c += 1
            if line == "Done High" or c>10:
                break
        Arduino.write(b'L')
        time.sleep(2)
        """
    #Arduino.write(b'D')
    #Arduino.close()
    cam.release()


In [98]:
cam = cv2.VideoCapture(0)



In [99]:
"""
Arduino = serial.Serial("COM5",57600)
img_counter = 0

while True:
    Arduino.write(b'R')
    line  = Arduino.readline().decode('utf-8').rstrip()
    if line == 'Startup is complete':
        break
"""


'\nArduino = serial.Serial("COM5",57600)\nimg_counter = 0\n\nwhile True:\n    Arduino.write(b\'R\')\n    line  = Arduino.readline().decode(\'utf-8\').rstrip()\n    if line == \'Startup is complete\':\n        break\n'

In [105]:
dirname1 = r'test'
img_counter = 0
r = 0
index1 = 0
a='#249794'
splash()
cam.open(0)
#Arduino.open()

Image Taken  [0]
Image Taken  [1]
Image Taken  [2]
Loading start_prog..
['papaya_images_0.png', 'papaya_images_1.png', 'papaya_images_2.png']
test\papaya_images_0.png
1
2


True

In [101]:
img_counter = 0
#main()


In [102]:
#cam.open(0)
#Arduino.open()